## Tokens on the Algorand Blockchain
#### 04.1 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2024-02-09 (started 2021-11-28)

* Opt-in and payments with tokens

## Setup
Starting with chapter 3.5, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()['last-round']

38993296

# ASA = Algorand Standard Assets
* All tokens (except the native ALGOs) on the Algorand blockchain are ASAs
* Every ASA has a unique index number.

### Example: USDC
* USDC is the second largest USD stablecoin
* See https://coinmarketcap.com/view/stablecoin/
* Unlike the largest one (Tether), USDC is US-regulated and audited
* USDC on the Testnet: https://testnet.explorer.perawallet.app/asset/10458941
* USDC on the Mainnet: https://explorer.perawallet.app/asset/31566704

## The opt-in requirement
* Every account needs to opt into *each* ASA it wants to hold.
* Same for Smart Contracts.

##### Why the opt-in requirement?
* Avoid spam transactions
* Ensure minimum balance

### Opt-in with the Pera Algo Wallet
* If you have not yet done so, opt into USDC on your MyAlgo account
    * Tap on `MyAlgo` > Tap on green `+` next to "Assets" > Find name or asset ID on list > Tap on `+` >  Approve transaction
    * Obtain free (test) USDC by following the link below

In [4]:
print('https://dispenser.testnet.aws.algodev.network/?account='+MyAlgo['public'])

https://dispenser.testnet.aws.algodev.network/?account=WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI


### Manual transfer in Pera Algo Wallet
* Send 10 USDC via QR code to your neighbour

##  Opt-in with Python
* `Alice` opts into USDC
    * She needs the index of the ASA – for USDC **on the TestNet** this is 10458941
    * She sends 0 units of the ASA to yourself
    * New transaction type `AssetTransferTxn` 

In [5]:
# Alice opts into USDC. 

# Step 1: prepare AssetTransferTxn

sp = algod_client.suggested_params()             # suggested params
usdc_index = 10458941                            # <-- get from the issuer of the coin
amt = int(0)                                     # <-- send 0 coins

txn = AssetTransferTxn(
    sender = Alice['public'],                    # <-- Alice sends ...
    sp=sp,
    receiver=Alice['public'],                    # <-- ... to herself
    amt=amt,
    index=usdc_index                             # <-- specify ASA
    )                               

In [6]:
# Step 2+3: sign and send
stxn = txn.sign(Alice['private'])               # Sign
txid = algod_client.send_transaction(stxn)      # Send
print(txid)

LSQOEHQLFNELM3HIS7GXN5DYWWMLFJQRLKG2WOHBF3TYDPUAA6BQ


In [7]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  38993300.
Waiting for round 38993300 to finish.
Transaction LSQOEHQLFNELM3HIS7GXN5DYWWMLFJQRLKG2WOHBF3TYDPUAA6BQ confirmed in round 38993301.


#### Alice gets some USDC
* Now that Alice has opted in, she can ask for some USDC
* Alternative address https://usdcfaucet.com

In [8]:
print('https://dispenser.testnet.aws.algodev.network/?account='+Alice['public'])

https://dispenser.testnet.aws.algodev.network/?account=ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU


#### Check balance on the web
* **Hint:** Scroll down to *Assets*

In [9]:
print(cred['explore_test']+'address/'+Alice['public'])

https://testnet.explorer.perawallet.app/address/ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU


#### Get the balance in Python

In [ ]:
algod_client.account_info(Alice['public'])

In [14]:
# Just the assets
algod_client.account_info(Alice['public'])['assets']

[{'amount': 279440275, 'asset-id': 10458941, 'is-frozen': False},
 {'amount': 250, 'asset-id': 592359981, 'is-frozen': False},
 {'amount': 0, 'asset-id': 592647685, 'is-frozen': False}]

In [15]:
# Potentially a very long list ... filter entry of interest
[asset for asset in algod_client.account_info(Alice['public'])['assets'] if asset['asset-id']==usdc_index]

[{'amount': 279440275, 'asset-id': 10458941, 'is-frozen': False}]

## ASA bookkeeping
Have a close look at Alice's holdings ...
* ASA are divisible
* The creator can decide the number of decimals
* Example: USDC has 6 decimals, see https://testnet.explorer.perawallet.app/asset/10458941

##### All amounts are specified in the *small unit*
* Holdings
* Transactions
* Smart contracts

##### ❗️ Different ASA can have a different number of decimals

#### Denoting 1 USDC
* USDC has 6 decimals
* Hence 1 USDC must be denoted as   $1 \cdot 10^6$   micro-USDC

In [16]:
# Three ways to write "1 Million" in Python
amt_1 = 1000000
amt_2 = 1_000_000       # Python ignores the underscore in numbers
amt_3 = int(1E6)        # Need int(), because 1E6 produces a float number

## Transfer coins

### Exercise: Python transfer to Bob 
* Transfer 10 USDC from `Alice` to `Bob`
* We already have all the elemens to do this
* Just do not forget to ... 

In [ ]:
# Your Python code goes here ...




In [ ]:
# More Python code may go here ...




#### Then check Bob's balance on the Web

In [17]:
print(cred['explore_test']+'address/'+Bob['public'])

https://testnet.explorer.perawallet.app/address/BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU
